In [1]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import LlamaCppEmbeddings
from langchain.llms import LlamaCpp
from langchain.schema.document import Document
from TendersWA.Preprocessing.Tender import Tender

In [15]:
# load some Tender
import os
os.chdir("/home/ucc/maxichat/Capstone/data/tender_raw")
t = Tender.load("ZPA980122.pickle")

In [3]:
llamaEmbeddings = LlamaCppEmbeddings(
    model_path="/home/ucc/maxichat/Capstone/models/llama/llama-2-7b-chat.Q5_K_S.gguf", 
    n_ctx = 256,
    verbose=False,
    n_threads = 64)                                                                 

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/ucc/maxichat/Capstone/models/llama/llama-2-7b-chat.Q5_K_S.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q5_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weigh

In [41]:
%%time
tender_vector_store = Chroma.from_documents(
    [Document(page_content=t.file_map["DESCRIPTION"])], 
    llamaEmbeddings) 

CPU times: user 5min 5s, sys: 46.9 ms, total: 5min 5s
Wall time: 19.2 s


In [4]:
from langchain.chains import RetrievalQA
llama_llm = LlamaCpp(
    model_path="/home/ucc/maxichat/Capstone/models/llama/llama-2-7b-chat.Q5_K_S.gguf")

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/ucc/maxichat/Capstone/models/llama/llama-2-7b-chat.Q5_K_S.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q5_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weigh

In [43]:
qa = RetrievalQA.from_chain_type(llm=llama_llm, chain_type="stuff", retriever=tender_vector_store.as_retriever())

In [44]:
qa.run("Summarize in three words")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2

llama_print_timings:        load time =  5541.21 ms
llama_print_timings:      sample time =     6.03 ms /     7 runs   (    0.86 ms per token,  1160.29 tokens per second)
llama_print_timings: prompt eval time = 122397.49 ms /   184 tokens (  665.20 ms per token,     1.50 tokens per second)
llama_print_timings:        eval time =  4770.97 ms /     7 runs   (  681.57 ms per token,     1.47 tokens per second)
llama_print_timings:       total time = 127252.51 ms


' Turf Maintenance Services'

In [64]:
from contextlib import contextmanager,redirect_stderr,redirect_stdout
from os import devnull

@contextmanager
def suppress_stdout_stderr():
    """A context manager that redirects stdout and stderr to devnull"""
    with open(devnull, 'w') as fnull:
        with redirect_stderr(fnull) as err, redirect_stdout(fnull) as out:
            yield (err, out)

In [11]:
def llama_topic(ref):
    t = Tender.load(ref) 
    print(t.file_map["DESCRIPTION"])
    return
    llamaEmbeddings = LlamaCppEmbeddings(
        model_path="/home/ucc/maxichat/Capstone/models/llama/llama-2-7b-chat.Q5_K_S.gguf", 
        n_ctx = 1024,
        verbose=False) 
    embs = Chroma.from_documents([Document(page_content=t.file_map["DESCRIPTION"])],
                                 llamaEmbeddings)
    llama_llm = LlamaCpp(
        model_path="/home/ucc/maxichat/Capstone/models/llama/llama-2-7b-chat.Q5_K_S.gguf")
    qa = RetrievalQA.from_chain_type(llm=llama_llm, 
                                     chain_type="stuff", 
                                     retriever=embs.as_retriever())
    print(qa.run("Summarize using five words or less. Do not use more than five words."))

In [12]:
%%time
import os
os.chdir("/home/ucc/maxichat/Capstone/data/tender_raw")
llama_topic("FINHRMIS0119AB")

HRMIS0119AB Request for Aggregated Buy
CPU times: user 0 ns, sys: 2.73 ms, total: 2.73 ms
Wall time: 2.29 ms


In [10]:
Tender.load("FINHRMIS0119AB").file_map["DESCRIPTION"]

'HRMIS0119AB Request for Aggregated Buy'